In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense, Input, BatchNormalization
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load training and testing data 

X_train = np.load('/content/drive/MyDrive/GTSRB/X_train.npy')
X_test = np.load('/content/drive/MyDrive/GTSRB/X_test.npy')
y_train = np.load('/content/drive/MyDrive/GTSRB/y_train.npy')
y_test = np.load('/content/drive/MyDrive/GTSRB/y_test.npy')

In [4]:
print("Shape of X_train : ", X_train.shape)
print("Shape of X_test : ", X_test.shape)
print("Shape of y_train : ", y_train.shape)
print("Shape of y_test : ", y_test.shape)

Shape of X_train :  (35289, 30, 30, 3)
Shape of X_test :  (3920, 30, 30, 3)
Shape of y_train :  (35289, 1)
Shape of y_test :  (3920, 1)


In [5]:
# Set basic parameters

BATCH_SIZE = X_train.shape[0]
ROWS = X_train.shape[1]
COLUMNS = X_train.shape[2]
CHANNELS = X_train.shape[3]
NUM_CLASSES = 43

In [6]:
# One-hot encoding for training and testing target vector

y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

print("Shape of y_train after one-hot encoding : ", y_train.shape)
print("Shape of y_test after one-hot encoding : ", y_test.shape)

Shape of y_train after one-hot encoding :  (35289, 43)
Shape of y_test after one-hot encoding :  (3920, 43)


In [8]:
# Normalizing feature vectors

(X_train, X_test) = (X_train/255.0, X_test/255.0)

In [9]:
# Set input shape

input_shape = (ROWS, COLUMNS, CHANNELS)

In [22]:
# Build the network

i = Input(shape=input_shape, name="Input Layer")

#CONV BLOCK 1
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name = 'Conv0')(i)
x = BatchNormalization(name='BatchNormalization0')(x)
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name='Conv1')(x)
x = MaxPool2D(pool_size=(2,2), name='MaxPool0')(x)

#CONV BLOCK 2
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name = 'Conv2')(x)
x = BatchNormalization(name='BatchNormalization1')(x)
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name='Conv3')(x)
x = MaxPool2D(pool_size=(2,2), name='MaxPool1')(x)

#CONV BLOCK 3
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name = 'Conv4')(x)
x = BatchNormalization(name='BatchNormalization2')(x)
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name='Conv5')(x)
x = MaxPool2D(pool_size=(2,2), name='MaxPool2')(x)


#DENSE LAYERS
x = Flatten(name='Flatten')(x)
x = Dropout(rate=0.2)(x)
x = Dense(units=1024, activation='relu', name='Dense0')(x)
x = Dense(units=512, activation='relu', name='Dense1')(x)
x = Dense(units=128, activation='relu', name='Dense2')(x)
output = Dense(units=43, activation='softmax', name='output')(x)

In [23]:
# Make a model from the network

model = Model(inputs=i, outputs=output, name="GTSRB_CLASSIFIER")
model.summary()

Model: "GTSRB_CLASSIFIER"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input Layer (InputLayer)     [(None, 30, 30, 3)]       0         
_________________________________________________________________
Conv0 (Conv2D)               (None, 30, 30, 32)        896       
_________________________________________________________________
BatchNormalization0 (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
Conv1 (Conv2D)               (None, 30, 30, 32)        9248      
_________________________________________________________________
MaxPool0 (MaxPooling2D)      (None, 15, 15, 32)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 15, 15, 32)        9248      
_________________________________________________________________
BatchNormalization1 (BatchNo (None, 15, 15, 32)   

In [24]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the network
EPOCHS = 25
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/25
1103/1103 [==============================] - 168s 151ms/step - loss: 3.1779 - accuracy: 0.1299 - val_loss: 2.4619 - val_accuracy: 0.2617
Epoch 2/25
1103/1103 [==============================] - 166s 151ms/step - loss: 2.4155 - accuracy: 0.2718 - val_loss: 2.1840 - val_accuracy: 0.3332
Epoch 3/25
1103/1103 [==============================] - 167s 152ms/step - loss: 2.1286 - accuracy: 0.3452 - val_loss: 2.0297 - val_accuracy: 0.3765
Epoch 4/25
1103/1103 [==============================] - 168s 152ms/step - loss: 1.9328 - accuracy: 0.3883 - val_loss: 1.9027 - val_accuracy: 0.4046
Epoch 5/25
1103/1103 [==============================] - 168s 152ms/step - loss: 1.7923 - accuracy: 0.4253 - val_loss: 1.7822 - val_accuracy: 0.4497
Epoch 6/25
1103/1103 [==============================] - 167s 152ms/step - loss: 1.6485 - accuracy: 0.4699 - val_loss: 1.6280 - val_accuracy: 0.4857
Epoch 7/25
1103/1103 [==============================] - 166s 150ms/step - loss: 1.5362 - accuracy: 0.5053 - val_